### PHASE TWO PROJECT -GROUP 3

### 1) PROJECT OVERVIEW
###      1.1 BUSINESS UNDERSTANDING 
###         1.1.1 Business Problem 

A company has decided to create a new movie studio, but they don’t know anything about creating movies. We have been charged with exploring what types of films are currently doing the best at the box office. We must then translate those findings into actionable insights that the head of the company's new movie studio can use to help decide what type of films to create.

### 1.1.2 Key Business Questions
* Which Movie Genres have the highest average rating?
* Which Movie Genres have the highest Number of votes?
* What impact does the runtime in minutes has on the average rating?
* How does language influence the average rating of movies?
* How does region influence the number of votes of movies?
* 

### 2) DATA UNDERSTANDING 

### 2.1 DATA PREPROCESSING
### 2.1.1 The Data 
To ensure comprehensive analysis of the business problem we retrieved data from the IMDB website (https://www.imdb.com/).The IMDB dataset has eight tables and for this particular project we will focus on three tables, namely 'movie_basics', 'movie_ratings' and 'movie_akas'. These three tables have share a column 'movie_id' which will be our Primary Key. From 'movie_basics' table we focus on 'Genres' and 'Runtime in Minutes'.From  'movie_ratings'  we focus on 'average ratings' and 'number of votes'. From the 'movie_akas' we focus on 'region' and 'language'.

### 2.1.2 DATA PREPARATION
This entails;
* Importing necessary libraries 
* Loading and Accessing of the data set
* Data Cleaning 

### A) Importing necessary libraries  

In [1]:
# importing necessary libraries 
import itertools
import numpy as np
import pandas as pd 
from numbers import Number
import sqlite3
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import pickle

## B) Acessing the Database 

In [2]:
# Connecting to the'im.db' database
conn = sqlite3.connect('im.db')

In [3]:
#accessing tables in the database 
q ='''
SELECT  name 
FROM sqlite_master 
WHERE type='table';
'''
pd.read_sql(q, conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


From the above we are able to tell the table  that are contained in our database 

In [4]:
#Viewing columns in the movie_basics table from the database
q = '''
SELECT *
FROM movie_basics
'''
pd.read_sql(q, conn)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [5]:
#Viewing columns in the movie_ratings table from the database
q = '''
SELECT *
FROM movie_ratings
'''
pd.read_sql(q, conn)

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


In [6]:
#Viewing columns in the movie_akas table from the database
q = '''
SELECT *
FROM movie_akas
'''
pd.read_sql(q, conn)

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0
...,...,...,...,...,...,...,...,...
331698,tt9827784,2,Sayonara kuchibiru,None,None,original,None,1.0
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,None,0.0
331700,tt9880178,1,La atención,None,None,original,None,1.0
331701,tt9880178,2,La atención,ES,None,None,None,0.0


In [8]:
### doing an inner join of movie_basics, movie_ratings and movie_akas
q = '''
SELECT * 
FROM movie_basics
JOIN movie_ratings
USING(movie_id)
JOIN movie_akas
USING(movie_id)
''' 
data = pd.read_sql(q, conn)
data

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,ordering,title,region,language,types,attributes,is_original_title
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,1,Sangharsh,IN,hi,None,alternative transliteration,0.0
1,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,2,Sunghursh,None,None,original,None,1.0
2,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,3,Sunghursh,IN,None,None,None,0.0
3,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,4,Sunghursh,IN,hi,None,alternative transliteration,0.0
4,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,5,Sungharsh,IN,hi,None,alternative spelling,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261801,tt9905462,Pengalila,Pengalila,2019,111.0,Drama,8.4,600,2,Sisterleaf,IN,en,None,literal English title,0.0
261802,tt9905462,Pengalila,Pengalila,2019,111.0,Drama,8.4,600,3,Pengalila,IN,None,None,None,0.0
261803,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,130.0,Drama,8.4,365,1,Padmavyuhathile Abhimanyu,None,None,original,None,1.0
261804,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,130.0,Drama,8.4,365,2,Pathmavyuhathile Abhimanyu,IN,ml,None,alternative spelling,0.0
